# Counter-Strike ML - Round Swing on Major Games

This notebook is a machine learning personal project by [sadhubby](https://github.com/sadhubby) that analyses professional Counter Strike games during the StarLadder Budapest 2025 Major Playoffs and calculates round swing on actions done across the map each round.

As a high-level example, this is similar to Chess.com's move analyzer that calculates the possibility of winning for either black or white based on the move done by the player.

The intended use of this machine learning software is to be able to analyze one's own personal games in Counter-Strike, whether through Valve's matchmaking or FaceIt's third-party matchmaking. This is so that the user will have numerical and graphical evidence of how great an impact a certain action is to the round as a whole. 

First, importing [pandas](https://pandas.pydata.org/) library for data analysis and [awpy](https://github.com/pnxenopoulos/awpy) for the gathering the data from recorded professional Counter Strike games during the Major. The video files are taken from [HLTV](https://www.hltv.org/), a website for Counter Strike News and Coverage. They include the recorded videos, called match demos, for every game done during the Major. 

In [1]:
import pandas as pd
from awpy import Demo

Before anything, it is first best to test this on a single map of a match. For the purpose of testing, Map 2 of the match between Vitality and FaZe Clan during the Grand Finals of StarLadder Budapest Major 2025.

Here, we first create a Demo instance of a match and parsing the instance.

In [2]:
dem = Demo("vitality-vs-faze-m2-dust2.dem")
dem.parse()

Then we now gather the rounds of the map into `rounds_df`

In [27]:
rounds_df = dem.rounds.to_pandas()


In [28]:
print(f"Total Rounds Played: {len(rounds_df)}")

Total Rounds Played: 16


Next, we gather the amount of ticks that has happened in a round. Counter Strike 2 live gameplay happens at Sub-Tick, their new, and quite unwanted feature that came with the port to the Source 2 Engine. However, for the purposes of this project, the ticks will be recorded at 64 ticks.

In [29]:
ticks = dem.ticks.to_pandas()
ticks['seconds_elapsed'] = ticks['tick'] / 64

In [15]:
round_5_ticks = ticks[ticks['round_num'] == 5]

In [17]:
round_5_ticks

,health,place,side,X,Y,Z,tick,steamid,name,round_num,seconds_elapsed
351540,100,,ct,258.159393,2480.553711,-120.976624,35175,76561198201620490,broky,5,549.609375
351541,100,,ct,334.368744,2433.733643,-120.262512,35175,76561198016255205,Twistzz,5,549.609375
351542,100,,t,-857.506531,-738.361328,122.189125,35175,76561198113666193,ZywOo,5,549.609375
351543,100,,t,-493.000000,-808.000000,108.663376,35175,76561197989744167,apEX,5,549.609375
351544,100,,t,-332.000000,-754.000000,78.977112,35175,76561197978835160,flameZ,5,549.609375
...,...,...,...,...,...,...,...,...,...,...,...
448995,77,OutsideLong,t,729.678528,-212.222473,9.031249,44920,76561197973140692,mezii,5,701.875000
448996,0,BombsiteB,ct,-1499.307129,2091.949219,-0.031250,44920,76561197989430253,karrigan,5,701.875000
448997,0,Hole,ct,-1280.000122,2671.929199,130.031250,44920,76561198178737429,jcobbb,5,701.875000
448998,0,BDoors,ct,-1209.692017,2309.067627,9.321785,44920,76561198068422762,frozen,5,701.875000


In [20]:
round_5_summary = round_5_ticks.groupby(['tick', 'seconds_elapsed', 'name'])['health'].sum().unstack()

In [21]:
round_5_summary

,name,Twistzz,ZywOo,apEX,broky,flameZ,frozen,jcobbb,karrigan,mezii,ropz
tick,seconds_elapsed,,,,,,,,,,
35175,549.609375,100,100,100,100,100,100,100,100,100,100
35176,549.625000,100,100,100,100,100,100,100,100,100,100
35177,549.640625,100,100,100,100,100,100,100,100,100,100
35178,549.656250,100,100,100,100,100,100,100,100,100,100
35179,549.671875,100,100,100,100,100,100,100,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...
44916,701.812500,0,84,0,100,0,0,0,0,77,31
44917,701.828125,0,84,0,100,0,0,0,0,77,31
44918,701.843750,0,84,0,100,0,0,0,0,77,31


Now, after having seen the rounds and ticks data frames and their respective columns, as well as see the parsing function work, we can now start creating the pipeline for the machine learning algorithm. 

Let us first settle the inputs and outputs. 
Our output, the outcome we want to see, is who will be the **winner** of a certain round given:
- the number of Counter-Terrorists (CT's) and Terrorist (T's) alive
- if the bomb is planted or not
- the time remaining in the round after certain events (such as eliminations, bomb plant)
- difference in equipment value (difference of the culmination of firepower and grenades from both CT's and T's)  